# PARAMETERS

###Update: 

**True** if you want to update all classifications 

**False** if you only want to add new classifications

In [1]:
update = False #@param {type:"boolean"}
fresh = True #@param {type:"boolean"}

# Installing GDAL

In [2]:
%%capture
!apt-get update
!apt-get install libgdal-dev -y
!apt-get install python-gdal -y
!apt-get install python-numpy python-scipy -y
!pip install rasterio
!pip install fiona
!pip install geopandas
import gdal 

# Importing Libraries

In [3]:
from tqdm.autonotebook import tqdm
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import rasterio
from descartes import PolygonPatch
from rasterio.plot import show
import matplotlib as mpl
import geopandas
import fiona

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

print("TF version:", tf.__version__)
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  """Entry point for launching an IPython kernel.


TF version: 2.2.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is available


# Loading Data from Drive





In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
from tensorflow.keras.models import load_model

!cp "/content/drive/Shared drives/SIO and E4E Mangroves /Data/Machine Learning/Models/mvnmv4_merced_bright.zip" .
!unzip mvnmv4_merced_bright.zip -A

#Set model location
model = "/content/mvnmv4_merced/"
model = load_model(model)

Archive:  mvnmv4_merced_bright.zip
caution: filename not matched:  -A


In [6]:
def retile(full_path):
    name = os.path.basename(full_path)
    file_string = "\"" + full_path + "\""
    name_string = "\"" + os.path.basename(full_path) + "\""

    print("Downloading {}".format(name))
    !cp {file_string} .

    !mkdir /content/images
    !mkdir /content/images/images

    print("Retiling {}".format(name))
    call = "gdal_retile.py -ps 256 256 -targetDir /content/images/images/ /content/" + name
    !{call}

    !rm {name}

In [7]:
#Since the original model outputs the values from the last dense layer (no final activation), we need to definte the sigmoid function for predicted class conditional probabilities
def sigmoid(x):
    return 1/(1 + np.exp(-x)) 

In [8]:
def classify_tiles(image_directory):
    print("Classifying tiles")
    #Read images using keras and split into batches
    image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
    data_gen = image_generator.flow_from_directory(directory=image_directory,
                                                        batch_size=32,
                                                        shuffle=False,
                                                        target_size=(256, 256))
    #Set up dataframe that will hold classifications
    column_names = ["prediction","p_0","p_1","filename"]
    result_df = pd.DataFrame(columns=column_names)

    #predict probabilities from model for the batches
    predictions = model.predict(data_gen)

    #associate filenames and classification for each prediction
    for i,prediction in tqdm(enumerate(predictions)):
        result_df.loc[i,"filename"] = data_gen.filenames[i]

        #calculating predictions 
        result_df.loc[i,"p_0"] = sigmoid(prediction[0])
        result_df.loc[i,"p_1"] = sigmoid(prediction[1])
        
        #getting final class prediction
        result_df.loc[i,"prediction"] = np.argmax(prediction)
    
    return result_df

In [9]:
def generate_probtiles(result_df):
    print("Generating tiles for probability plot")
    for index, sample in tqdm(result_df.iterrows()):
        #loading original image
        original = os.path.abspath(os.path.join("images", sample["filename"]))
        img = rasterio.open(original)

        #creating new raster mask with pixel values of conditional probability
        mask = sample["p_0"] * np.ones(shape=(img.width, img.height))

        #saving file output to new file
        filename = "prob_" + os.path.basename(sample["filename"])
        output = os.path.abspath(os.path.join("images", os.path.dirname(sample["filename"]), filename))
        #creates new file with projection of past image
        with rasterio.open(output,'w',driver='GTiff',height=img.height,width=img.width,count=1,dtype=mask.dtype,crs='+proj=latlong',transform=img.transform,) as dst:dst.write(mask, 1)

In [10]:
def move_tiles(result_df):
    print("Moving M/NM Tiles into folders")
    dest_folders = []
    #Organize tiles into folders
    for index, row in result_df.iterrows():
        cur_file = "/content/images/" + row['filename']
        cur_file = cur_file.replace("jpg","tif",2)
        classification = row['prediction'] 

        #set destination folder, and creates the folder if it doesn't exist
        dest_folder = os.path.join(os.path.abspath(image_directory),str(classification))
        dest_folders.append(dest_folder)
        if os.path.exists(dest_folder) == False:
            os.mkdir(dest_folder)
        dest = os.path.join(dest_folder,os.path.basename(cur_file))

        #moves file
        src = cur_file
        os.rename(src, dest)

In [11]:
def fix_shp(filename):
    shp = geopandas.read_file(filename)
    for index, feature in tqdm(shp.iterrows()):
        if feature["DN"] == 0:
            shp.drop(index, inplace=True)
    shp.to_file(filename)
    return shp

In [12]:
def create_files(full_path):
    #remove any already existing files in the path

    file = os.path.basename(full_path)
    folder = os.path.dirname(full_path)
    name = os.path.splitext(file)[0]
    dest_folder = os.path.join(folder,name+'_classifications/')

    mtif_name = 'm_' + name + '.tif'
    mshp_name = 'm_' + name + '.shp'

    nmtif_name = 'nm_' + name + '.tif'
    nmshp_name = 'nm_' + name + '.shp'

    ptif_name =  'prob_' + name + '.tif'
    #plot_name =  'plot_' + name + '.png'

    print("Creating Orthomosaics")
    #recombines classified tiles for each class
    !gdal_merge.py -o /content/{nmtif_name} /content/images/1/*
    !rm /content/images/1/*.tif
    !gdal_merge.py -o /content/{mtif_name} /content/images/0/*
    !rm /content/images/0/*.tif

    #probability tiles remain unmoved, so just get all the leftover tiles
    !gdal_merge.py -o /content/{ptif_name} /content/images/images/*
    !rm /content/images/images/*.tif

    !gdal_polygonize.py /content/{nmtif_name} -f "ESRI Shapefile" -b 4 {nmshp_name}
    !gdal_polygonize.py /content/{mtif_name} -f "ESRI Shapefile" -b 4 {mshp_name}

    print("Creating plots and saving them")



    #nonmangrove
    #src = rasterio.open(nmtif_name)
    shp = fix_shp(nmshp_name)

    '''
    fig, ax = plt.subplots()
    rasterio.plot.show(src, ax=ax)
    shp.plot(ax=ax, facecolor='green', edgecolor='red', alpha=0.25)
    fig.savefig("nm" + plot_name)

    del shp
    del src
    del fig
    del ax 
    '''
    #mangrove
    #src = rasterio.open(mtif_name)
    shp = fix_shp(mshp_name)

    '''
    fig, ax = plt.subplots()
    rasterio.plot.show(src, ax=ax)
    shp.plot(ax=ax, facecolor='green', edgecolor='red', alpha=0.25)
    fig.savefig("m" + plot_name)

    del shp
    del src
    del fig
    del ax 

    fig, ax = plt.subplots()
    src = rasterio.open(ptif_name)
    rasterio.plot.show(src,ax=ax,cmap='viridis')
    fig.savefig("prob" + plot_name)

    del shp
    del src
    del fig
    del ax 
    '''

    !mkdir {"\"" + dest_folder + "\""}

    mtif_name = 'm_' + name + '.tif'
    mshp_name = 'm_' + name + '.shp'
    mshx_name = 'm_' + name + '.shx'
    mdbf_name = 'm_' + name + '.dbf'
    mprj_name = 'm_' + name + '.prj'
    mcpg_name = 'm_' + name + '.cpg'

    mtif_dest = os.path.join(dest_folder, mtif_name)
    mshp_dest = os.path.join(dest_folder, mshp_name)
    mshx_dest = os.path.join(dest_folder, mshx_name)
    mdbf_dest = os.path.join(dest_folder, mdbf_name)
    mprj_dest = os.path.join(dest_folder, mprj_name)
    mcpg_dest = os.path.join(dest_folder, mcpg_name)

    print("Uploading Mangrove Files")

    !cp {"\"" + mtif_name + "\""} {"\"" + mtif_dest + "\""}
    !cp {"\"" + mshp_name + "\""} {"\"" + mshp_dest + "\""}
    !cp {"\"" + mshp_name + "\""} {"\"" + mshp_dest + "\""}
    !cp {"\"" + mshp_name + "\""} {"\"" + mshp_dest + "\""}
    !cp {"\"" + mshp_name + "\""} {"\"" + mshp_dest + "\""}
    !cp {"\"" + mshp_name + "\""} {"\"" + mshp_dest + "\""}

    nmtif_name = 'nm_' + name + '.tif'
    nmshp_name = 'nm_' + name + '.shp'
    nmshx_name = 'nm_' + name + '.shx'
    nmdbf_name = 'nm_' + name + '.dbf'
    nmprj_name = 'nm_' + name + '.prj'
    nmcpg_name = 'nm_' + name + '.cpg'

    nmtif_dest = os.path.join(dest_folder, nmtif_name)
    nmshp_dest = os.path.join(dest_folder, nmshp_name)
    nmshx_dest = os.path.join(dest_folder, nmshx_name)
    nmdbf_dest = os.path.join(dest_folder, nmdbf_name)
    nmprj_dest = os.path.join(dest_folder, nmprj_name)
    nmcpg_dest = os.path.join(dest_folder, nmcpg_name)

    print("Uploading Non-Mangrove Files")


    !cp {"\"" + nmtif_name + "\""} {"\"" + nmtif_dest + "\""}
    !cp {"\"" + nmshp_name + "\""} {"\"" + nmshp_dest + "\""}
    !cp {"\"" + nmshx_name + "\""} {"\"" + nmshx_dest + "\""}
    !cp {"\"" + nmdbf_name + "\""} {"\"" + nmdbf_dest + "\""}
    !cp {"\"" + nmprj_name + "\""} {"\"" + nmprj_dest + "\""}
    !cp {"\"" + nmcpg_name + "\""} {"\"" + nmcpg_dest + "\""}

    ptif_name =  'prob_' + name + '.tif'
    plot_name =  'plot_' + name + '.png'

    ptif_dest = os.path.join(dest_folder, ptif_name)
    #plot_dest = os.path.join(dest_folder, plot_name)

    print("Uploading Other Files")

    !cp {"\"" + ptif_name + "\""} {"\"" + ptif_dest + "\""}
    #!cp {"\"" + "nm" + plot_name + "\""} {"\"" + "nm" + plot_dest + "\""}
    #!cp {"\"" + "m" + plot_name + "\""} {"\"" + "m" + plot_dest + "\""}
    #!cp {"\"" + "prob" + plot_name + "\""} {"\"" + "prob" + plot_dest + "\""}


In [13]:
def run(full_path,image_directory):
    !rm *.tif
    !rm *.png
    !rm *.cpg
    !rm *.dbf
    !rm *.prj
    !rm *.shp
    !rm *.shx

    !rm -rf images
    retile(full_path)
    result_df = classify_tiles(image_directory)
    generate_probtiles(result_df)
    move_tiles(result_df)
    del result_df
    create_files(full_path)

In [14]:
columns = ["filename","filesize (GB)","last_updated","full_path"]
stats_pkl = "/content/drive/Shared drives/SIO and E4E Mangroves /Data/Classification_statistics.pkl"

if fresh:
    class_stats = pd.DataFrame(columns=columns)
else:
    class_stats = pd.read_pickle(stats_pkl)

file_list = []
i = 0
for root, dirs, files in os.walk("/content/drive/Shared drives/SIO and E4E Mangroves /Data/Orthomosaics/"):
    for file in files:
        if not(("dem" in file) or ("DEM" in file)) and (file.endswith(".tif")):
            if file not in class_stats["filename"].to_list():
                full_path = os.path.join(root,file)
                file_size = os.path.getsize(full_path)/(1024*1024*1024)
                df = pd.DataFrame({"filename":file,"full_path":full_path,"filesize (GB)":file_size}, index=[i] ,columns=columns)
                class_stats = class_stats.append(df)
            i += 1

In [ ]:
from datetime import datetime

stats_location = "/content/drive/Shared drives/SIO and E4E Mangroves /Data/Classification_statistics.xlsx"

for index, row in class_stats.iterrows():
    image_directory = "/content/images"
    full_path = row["full_path"]
    model = "/content/mvnmv4_merced/"
    model = load_model(model)
    if update: 
        if row.isnull().values.any():
            run(full_path,image_directory)
            class_stats.loc[index, "last_updated"] = datetime.now()
            class_stats.to_excel(stats_location)
            pd.to_pickle(class_stats, stats_pkl)
        else:
            continue
    else: 
        run(full_path,image_directory)
        class_stats.loc[index, "last_updated"] = datetime.now()
        class_stats.to_excel(stats_location)
        pd.to_pickle(class_stats, stats_pkl)


rm: cannot remove '*.png': No such file or directory
Retiling skr_2019-05_site01_120m_RGB.tif
0...10...20...30...40...50...60...70...80...90...100 - done.
Classifying tiles
Found 13098 images belonging to 1 classes.



Generating tiles for probability plot



Moving M/NM Tiles into folders
Creating Orthomosaics
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output nm_skr_2019-05_site01_120m_RGB.shp of format ESRI Shapefile.
0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output m_skr_2019-05_site01_120m_RGB.shp of format ESRI Shapefile.
0...10...20...30...40...50...60...70...80...90...100 - done.
Creating plots and saving them



mkdir: cannot create directory ‘/content/drive/Shared drives/SIO and E4E Mangroves /Data/Orthomosaics/2019-05 Sian Ka'an Reserve/skr_2019-05_site01_120m_RGB_classifications/’: File exists
Uploading Mangrove Files
Uploading Non-Mangrove Files
Uploading Other Files
rm: cannot remove '*.png': No such file or directory
Retiling skr_2019-05_site04_120m_RGB.tif
0...10...20...30...40...50...60...70...80...90...100 - done.
Classifying tiles
Found 10948 images belonging to 1 classes.



Generating tiles for probability plot



Moving M/NM Tiles into folders
Creating Orthomosaics
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output nm_skr_2019-05_site04_120m_RGB.shp of format ESRI Shapefile.
0...10...20...30...40...50...60...70...80...90...100 - done.
Creating output m_skr_2019-05_site04_120m_RGB.shp of format ESRI Shapefile.
0...10...20...30...40...50...60...70...80...90...100 - done.
Creating plots and saving them



Uploading Mangrove Files
Uploading Non-Mangrove Files
Uploading Other Files
rm: cannot remove '*.png': No such file or directory
Retiling skr_2019-05_site06_120m_RGB.tif
0...10...20...30...40...50...60...70...80...90...100 - done.
Classifying tiles
Found 24786 images belonging to 1 classes.



Generating tiles for probability plot



Moving M/NM Tiles into folders
Creating Orthomosaics
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
ERROR 3: Free disk space available is 10036051968 bytes, whereas 12954428392 are at least necessary. You can disable this check by defining the CHECK_DISK_FREE_SPACE configuration option to FALSE.
Creation failed, terminating gdal_merge.
Creating output nm_skr_2019-05_site06_120m_RGB.shp of format ESRI Shapefile.
0..